## <font face="仿宋">课程说明：

- 体验课内容节选自[《2025大模型Agent智能体开发实战》(夏季班)](https://ix9mq.xetslk.com/s/2lPSMo)完整版付费课程

&emsp;&emsp;体验课时间有限，若想深度学习大模型技术，欢迎大家报名由我主讲的[《2025大模型Agent智能体开发实战》(夏季班)](https://ix9mq.xetslk.com/s/2lPSMo)

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506171642034.jpg" alt="bef0897853f861af5f4211442be446b" style="zoom:15%;" />

**[《2025大模型Agent智能体开发实战》(夏季班)](https://ix9mq.xetslk.com/s/2lPSMo)为【100+小时】体系大课，总共20大模块精讲精析，零基础直达大模型企业级应用！**

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506172010074.png" alt="a55d48e952ed59f8d93e050594843bc" style="zoom:33%;" />

部分项目成果演示

In [3]:
from IPython.display import Video

- **MateGen项目演示**

In [4]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/MG%E6%BC%94%E7%A4%BA%E8%A7%86%E9%A2%91.mp4", width=800, height=400)

- **智能客服项目演示**

In [5]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/%E6%99%BA%E8%83%BD%E5%AE%A2%E6%9C%8D%E6%A1%88%E4%BE%8B%E8%A7%86%E9%A2%91.mp4", width=800, height=400)

- **Dify项目演示**

In [6]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/2f1b47f42c65fd59e8d3a83e6cb9f13b_raw.mp4", width=800, height=400)

- **LangChain&LangGraph搭建Multi-Agnet**

In [7]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/%E5%8F%AF%E8%A7%86%E5%8C%96%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90Multi-Agent%E6%95%88%E6%9E%9C%E6%BC%94%E7%A4%BA%E6%95%88%E6%9E%9C.mp4", width=800, height=400)

此外，若是对大模型底层原理感兴趣，也欢迎报名由我和菜菜老师共同主讲的[《2025大模型原理与实战课程》(夏季班)](https://ix9mq.xetslk.com/s/3VITgV)

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506171650709.png" alt="4a11b7807056e9f5b281278c0e37dad" style="zoom:20%;" />

**两门大模型课程夏季班目前上新特惠+618年中钜惠双惠叠加，合购还有更多优惠哦~<span style="color:red;">详细信息扫码添加助教，回复“大模型”，即可领取课程大纲&查看课程详情👇</span>**

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506171644321.jpeg" alt="img" style="zoom: 33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506101749045.png" alt="6d9391e440ee8df1466cef1bce40705" style="zoom:50%;" />

---

# <center> LangChain快速入门与Agent开发实战
# <center> Part 8.LangChain RAG知识库检索系统开发实战

## 一、LangChain 实现本地知识库问答

&emsp;&emsp;供`Agents`在处理复杂任务的某个阶段使用，这其实是一种更为复杂的应用架构——Agent + RAG。

## 1.1 RAG基础概念入门


&emsp;&emsp;假设现在我们有一个偌大的知识库，当想从该知识库中去检索最相关的内容时，最简单的方法是：接收到一个查询（Query），就直接在知识库中进行搜索。这种做法其实是可行的，但存在两个关键的问题：
1. 假设提问的Query的答案出现在一篇文章中，去知识库中找到一篇与用户输入相关的文章是很容易的，但是我们将检索到的这整篇文章直接放入`Prompt`中并不是最优的选择，因为其中一定会包含非常多无关的信息，而无效信息越多，对大模型后续的推理影响越大。

2. 任何一个大模型都存在最大输入的Token限制，一个流程中可能涉及多次检索，每次检索都会产生相应的上下文，无法容纳如此多的信息。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202404101639353.png" width=80%></div>

&emsp;&emsp;解决上述两个问题的方式是：把存放着原始数据的知识库（Knowledge）中的每一个raw data，切分成一个一个的小块，这些小块可以是一个段落，也可以是数据库中某个索引对应的值。这个切分过程被称为“分块”（chunking），如下述流程所示：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202404101759694.png" width=80%></div>

&emsp;&emsp;以第一个原始数据为例（raw data 1），通过一些特定的方法进行切分，一个完整的内容会被分割成 chunk1 ~ chunk4。采取相同的方法，继续对`raw data 2`、`raw data 3`直至`raw data n`进行切分。完成这一过程后，我们最终得到的是一个充满分块数据（chunks）的新的知识库（repository），其中每一项都是一个单独的chunk。例如，如果原始文档共有10个，那么经过切分，可能会产生出100个chunks。

&emsp;&emsp;完成这一转化后，当再次接收到一个查询（Query）时，就会在更新后的知识库（repository）中进行搜索，这时检索的范围就不再是某个完整的文档，而是其中的某一个部分，返回的是一个或多个特定的chunk，这样返回的信息量就会更小且更精确。随后，这些被检索到的chunk会被加入到Prompt中，作为上下文信息与用户原始的Query共同输入到大模型进行处理，以生成最终的回答。

&emsp;&emsp;在上述将原始数据（raw data）转化为chunk的过程中，就会包含构建RAG的第一部分开发工作：这包括如果做数据清洗，如去除停用词、标点符号等。此外，还涉及如何选择合适的`split`方法来进行数据切分的一系列技术。

&emsp;&emsp;接下来面临的问题是，尽管所有数据已经被切割成一个个`chunk`，其存储形式还是以字符串形式存在，如果想从`repository`中匹配到与输入的query相关的chunks，比较两句话是否相似，看一句话中相同字有几个，这显然是行不通的。我们需要获取的是句子所蕴含的深层含义，而非仅仅是表面的字面相似度。因此，大家也能想到，在NLP中去计算文本相似度的有效的方法就是Embedding，即将这些chunks转换成向量（vector）形式。所以流程会丰富如下：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202404101823060.png" width=80%></div>

&emsp;&emsp;如上所示，解决搜索效率和计算相似度优化算法的答案就是：向量数据库。同时也产生了构建RAG的第三部分工作：我们要去了解和学习如何选择、使用向量数据库。

&emsp;&emsp;最终整体流程就如上图所示，一个基础的RAG架构会只要包含以下几方面的开发工作：

1. 如何将原始数据转化成chunks；
2. 如何将chunks转化成Vector；
3. 如何选择计算向量相似度的算法；
4. 如何利用向量数据库提升搜索效率；
5. 如何把找到的chunks与原始query拼接在一起，产生最终的Prompt；

&emsp;&emsp;而上述流程，其实更像是一个自由拼接的结果，比如不同的文档类型可以选择不同的文档解析器，也可以选择不同的`Vector`数据库，甚至可以自由选择`Embedding`模型和`Vector`数据库的组合。其自由程度非常高，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506091835680.png" width=80%></div>

## 1.2 从零到一搭建小型RAG系统入门教程

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506171955565.png" alt="image-20250617195516494" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506171816930.png" alt="57d34aa34ec04a98b020018c55b242d" style="zoom:33%;" />

## 1.3 实战：支持 PDF 文件解析的智能RAG 问答

&emsp;&emsp;理解了在`LangChain`中构建`RAG`的基本原理后，我们就可以开始动手实践了。接下来的案例中，我们通过 `Streamlit` 前端界面，结合 `LangChain` 框架 与 `DashScope` 向量嵌入服务，实现了一个轻量化的 `RAG（Retrieval-Augmented Generation）` 智能问答系统，支持上传多个 `PDF` 文档，系统将自动完成文本提取、分块、向量化，并构建基于 `FAISS` 的检索数据库。用户随后可以在页面中输入任意问题，系统会调用大语言模型（如 `DeepSeek-Chat`）对 `PDF` 内容进行语义理解和回答生成。

&emsp;&emsp;其完整代码如下所示：

In [6]:
# ! pip install streamlit PyPDF2 dashscope faiss-cpu

```python
    import streamlit as st
    from PyPDF2 import PdfReader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_community.vectorstores import FAISS
    from langchain.tools.retriever import create_retriever_tool
    from langchain.agents import AgentExecutor, create_tool_calling_agent
    from langchain_community.embeddings import DashScopeEmbeddings
    from langchain.chat_models import init_chat_model
    import os
    from dotenv import load_dotenv 
    load_dotenv(override=True)


    DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
    dashscope_api_key = os.getenv("dashscope_api_key")

    os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


    embeddings = DashScopeEmbeddings(
        model="text-embedding-v1", dashscope_api_key=dashscope_api_key
    )

    def pdf_read(pdf_doc):
        text = ""
        for pdf in pdf_doc:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text


    def get_chunks(text):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = text_splitter.split_text(text)
        return chunks

    def vector_store(text_chunks):
        vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
        vector_store.save_local("faiss_db")

    def get_conversational_chain(tools, ques):
        llm = init_chat_model("deepseek-chat", model_provider="deepseek")
        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """你是AI助手，请根据提供的上下文回答问题，确保提供所有细节，如果答案不在上下文中，请说"答案不在上下文中"，不要提供错误的答案""",
            ),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        
        tool = [tools]
        agent = create_tool_calling_agent(llm, tool, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)
        
        response = agent_executor.invoke({"input": ques})
        print(response)
        st.write("🤖 回答: ", response['output'])

    def check_database_exists():
        """检查FAISS数据库是否存在"""
        return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")

    def user_input(user_question):
        # 检查数据库是否存在
        if not check_database_exists():
            st.error("❌ 请先上传PDF文件并点击'Submit & Process'按钮来处理文档！")
            st.info("💡 步骤：1️⃣ 上传PDF → 2️⃣ 点击处理 → 3️⃣ 开始提问")
            return
        
        try:
            # 加载FAISS数据库
            new_db = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)
            
            retriever = new_db.as_retriever()
            retrieval_chain = create_retriever_tool(retriever, "pdf_extractor", "This tool is to give answer to queries from the pdf")
            get_conversational_chain(retrieval_chain, user_question)
            
        except Exception as e:
            st.error(f"❌ 加载数据库时出错: {str(e)}")
            st.info("请重新处理PDF文件")

    def main():
        st.set_page_config("🤖 LangChain B站公开课 By九天Hector")
        st.header("🤖 LangChain B站公开课 By九天Hector")
        
        # 显示数据库状态
        col1, col2 = st.columns([3, 1])
        
        with col1:
            if check_database_exists():
            pass
            else:
                st.warning("⚠️ 请先上传并处理PDF文件")
        
        with col2:
            if st.button("🗑️ 清除数据库"):
                try:
                    import shutil
                    if os.path.exists("faiss_db"):
                        shutil.rmtree("faiss_db")
                    st.success("数据库已清除")
                    st.rerun()
                except Exception as e:
                    st.error(f"清除失败: {e}")

        # 用户问题输入
        user_question = st.text_input("💬 请输入问题", 
                                    placeholder="例如：这个文档的主要内容是什么？",
                                    disabled=not check_database_exists())

        if user_question:
            if check_database_exists():
                with st.spinner("🤔 AI正在分析文档..."):
                    user_input(user_question)
            else:
                st.error("❌ 请先上传并处理PDF文件！")

        # 侧边栏
        with st.sidebar:
            st.title("📁 文档管理")
            
            # 显示当前状态
            if check_database_exists():
                st.success("✅ 数据库状态：已就绪")
            else:
                st.info("📝 状态：等待上传PDF")
            
            st.markdown("---")
            
            # 文件上传
            pdf_doc = st.file_uploader(
                "📎 上传PDF文件", 
                accept_multiple_files=True,
                type=['pdf'],
                help="支持上传多个PDF文件"
            )
            
            if pdf_doc:
                st.info(f"📄 已选择 {len(pdf_doc)} 个文件")
                for i, pdf in enumerate(pdf_doc, 1):
                    st.write(f"{i}. {pdf.name}")
            
            # 处理按钮
            process_button = st.button(
                "🚀 提交并处理", 
                disabled=not pdf_doc,
                use_container_width=True
            )
            
            if process_button:
                if pdf_doc:
                    with st.spinner("📊 正在处理PDF文件..."):
                        try:
                            # 读取PDF内容
                            raw_text = pdf_read(pdf_doc)
                            
                            if not raw_text.strip():
                                st.error("❌ 无法从PDF中提取文本，请检查文件是否有效")
                                return
                            
                            # 分割文本
                            text_chunks = get_chunks(raw_text)
                            st.info(f"📝 文本已分割为 {len(text_chunks)} 个片段")
                            
                            # 创建向量数据库
                            vector_store(text_chunks)
                            
                            st.success("✅ PDF处理完成！现在可以开始提问了")
                            st.balloons()
                            st.rerun()
                            
                        except Exception as e:
                            st.error(f"❌ 处理PDF时出错: {str(e)}")
                else:
                    st.warning("⚠️ 请先选择PDF文件")
            
            # 使用说明
            with st.expander("💡 使用说明"):
                st.markdown("""
                **步骤：**
                1. 📎 上传一个或多个PDF文件
                2. 🚀 点击"Submit & Process"处理文档
                3. 💬 在主页面输入您的问题
                4. 🤖 AI将基于PDF内容回答问题
                
                **提示：**
                - 支持多个PDF文件同时上传
                - 处理大文件可能需要一些时间
                - 可以随时清除数据库重新开始
                """)

    if __name__ == "__main__":
        main()
```

基于此，我们能够实现：

* **LangChain 的多模块能力**（向量搜索 + Agent工具）
* **Streamlit 前端交互**
* **FAISS 向量数据库**
* **DashScope Embedding + DeepSeek 模型接入**
* 并完成了完整的 RAG（检索增强生成）流程

以下是各部分功能实现代码讲解：

#### 🔧 1. 导入库 & 环境初始化

```python
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
...
load_dotenv(override=True)
```

* `Streamlit` 用于构建网页界面。
* `PyPDF2` 用来读取 PDF 文本。
* `load_dotenv()` 加载 `.env` 中的 API Key，例如：

  ```dotenv
  DEEPSEEK_API_KEY=sk-xxx
  DASHSCOPE_API_KEY=xxx
  ```

---

#### 🔐 2. 加载 API 密钥与设置环境变量

```python
DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
dashscope_api_key = os.getenv("dashscope_api_key")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
```

* 从环境变量中读取 DashScope 和 DeepSeek API。
* 设置 `KMP_DUPLICATE_LIB_OK` 避免某些 MKL 多线程报错。

---

#### 🧠 3. 初始化向量 Embedding 模型

```python
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=dashscope_api_key
)
```

* 用阿里云 DashScope 提供的 `text-embedding-v1` 将文本转为向量表示，用于相似度搜索。

---

#### 📄 4. 处理 PDF 文本与向量化逻辑

```python
def pdf_read(pdf_doc):
    ...
def get_chunks(text):
    ...
def vector_store(text_chunks):
    ...
```

* `pdf_read`：逐页读取 PDF 内容并拼接。
* `get_chunks`：将长文本切片为多个段落（chunk），每段 1000 字，重叠 200 字。
* `vector_store`：用 FAISS 建立向量索引，并保存到本地 `faiss_db/`。

---

#### 🔁 5. Agent对话链 + 工具调用（核心 RAG）

```python
def get_conversational_chain(tools, ques):
    llm = init_chat_model("deepseek-chat", model_provider="deepseek")
    ...
    agent_executor = AgentExecutor(...)
    response = agent_executor.invoke({"input": ques})
    ...
```

* 初始化 DeepSeek 模型为 Agent。
* 使用 LangChain 的 `create_tool_calling_agent` 构造 Agent，输入：

  * prompt（你设定的系统角色）
  * 工具（retriever 工具）
* `AgentExecutor.invoke`：LangChain 自动判断是否调用工具，完成“读取上下文 → 查询 → 回答”流程。

---

#### 🔍 6. 用户提问逻辑（调用 FAISS）

```python
def user_input(user_question):
    ...
    new_db = FAISS.load_local("faiss_db", embeddings, ...)
    retriever = new_db.as_retriever()
    retrieval_chain = create_retriever_tool(retriever, "pdf_extractor", ...)
    get_conversational_chain(retrieval_chain, user_question)
```

* 加载本地 FAISS 向量库；
* 将其转为 LangChain 的检索工具；
* 交由 Agent 调用完成回答。

---

#### 🧠 7. 检查数据库是否存在

```python
def check_database_exists():
    return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")
```

简单检查本地是否已有向量化数据。

---

#### 🌐 8. 主界面逻辑（Streamlit）

```python
def main():
    st.set_page_config(...)
    ...
```

* 页面标题与界面配置。
* `st.columns` 分栏：左边显示提示，右边放置“清空数据库”按钮。
* 主输入框：`st.text_input("请输入问题")`

  * 只有当数据库存在时才能提问。
* 侧边栏：

  * PDF 上传器；
  * 提交按钮（处理上传的 PDF → 分片 → 向量化 → 存储）。

---

#### 🎯 9. 提交 PDF 后执行的逻辑

```python
if process_button:
    raw_text = pdf_read(pdf_doc)
    ...
    text_chunks = get_chunks(raw_text)
    vector_store(text_chunks)
```

* 当点击“提交并处理”后：

  1. 读取上传的 PDF；
  2. 切片文本；
  3. 向量化入库；
  4. 弹出气球提示，并 `st.rerun()` 刷新页面状态。

---

#### 📎 项目结构总结

| 模块                | 说明                             |
| ----------------- | ------------------------------ |
| 🧾 PDF解析          | 读取用户上传的 PDF                    |
| ✂️ 文本切片           | 按段落分割内容                        |
| 📊 向量化            | DashScope Embedding + FAISS 建库 |
| 🔁 查询接口           | 用户输入 → 召回相关 chunk              |
| 🤖 DeepSeek Agent | 调用检索工具并给出回答                    |
| 💻 UI层            | Streamlit 实现全部交互               |

&emsp;&emsp;其中LangChain RAG核心功能相关代码如下：

- Step 1：PDF 文件上传与文本提取

&emsp;&emsp;使用 `st.file_uploader()` 组件支持多文件上传，并通过 `PyPDF2.PdfReader` 对每页内容进行提取，组合为整体文本。

```python
    def pdf_read(pdf_doc):
        text = ""
        for pdf in pdf_doc:
            pdf_reader = PdfReader(pdf)
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
```

- Step 2：文本分块与向量数据库构建

&emsp;&emsp;使用 `RecursiveCharacterTextSplitter` 将长文档切割为固定长度（1000字）+ 重叠（200字）的小块，将文本块通过 `DashScopeEmbeddings` 嵌入为向量，使用 `FAISS` 本地存储向量数据库。


```python
    chunks = text_splitter.split_text(text)
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    vector_store.save_local("faiss_db")
```


- Step 3：用户提问与语义检索

&emsp;&emsp;通过 `Streamlit` 获取用户输入问题，如果向量数据库存在，则加载 `FAISS` 检索器，使用 `create_retriever_tool()` 构建 `LangChain` 工具，交由 `AgentExecutor` 执行，自动调用检索器并生成答案。

```python
    retrieval_chain = create_retriever_tool(retriever, ...)
    agent = create_tool_calling_agent(llm, [retrieval_chain], prompt)
    response = agent_executor.invoke({"input": ques})
```

&emsp;&emsp;完整的代码已经上传至百度网盘中的`langchain_rag.py`文件中，大家可以扫描下方二维码免费领取

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506172002839.png" alt="image-20250617200256788" style="zoom:33%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506102031014.png" alt="6d9391e440ee8df1466cef1bce40705" style="zoom:50%;" />

&emsp;&emsp;项目运行效果如下所示：

In [2]:
from IPython.display import Video

In [3]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/LangChain%20RAG.mp4", width=800, height=400)